## Command imports

In [103]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


## Import mse IDs 

In [104]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1343"]
results_dict = {"mseID":["lesion number and type",
                         "number of subcortical lesions",
                         "number of juxtacortical lesions",
                         "number of periventricular lesions",
                         "number of infratentorial lesions",
                         "number of errors"]}
print subjects[:5]

['mse1343']


## Set variables to lesion/segmentation files

In [105]:
for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[0]):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[0]):
    print seg_file
    
print seg_file, les_file

/data/henry7/PBR/subjects/mse1343/lesions_manual/ms69-mse1343-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1343/masks/ms69-mse1343-002-AX_T1_3D_IRSPGR/segmentation.nii.gz
/data/henry7/PBR/subjects/mse1343/masks/ms69-mse1343-002-AX_T1_3D_IRSPGR/segmentation.nii.gz /data/henry7/PBR/subjects/mse1343/lesions_manual/ms69-mse1343-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz


## Obtain affines 

In [106]:
les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape

les_affine = les_img.get_affine()
print les_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 150)
(256, 256, 256)
[[  -0.9375        0.            0.          118.36100006]
 [   0.            0.9375        0.          -74.62950134]
 [   0.            0.            1.          -27.06119919]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          126.36100769]
 [   0.            0.            1.          -82.62950134]
 [   0.           -1.            0.          182.938797  ]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          126.36100769]
 [  -0.           -0.           -1.          182.938797  ]
 [   0.            1.            0.           82.62950134]
 [   0.            0.            0.            1.        ]]


## set labels for coordinates

In [107]:
#lesion labels
les_labels, n_les_labels = label(les_data==[1])

#segmentation labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

## debugging tools for labels

In [108]:
print n_les_labels #prints number of lesion labels

print n_seg_brainstem_labels #prints number of labels in brainstem

print np.bincount(les_labels.ravel()) #prints volumes of lesions

print np.bincount(seg_lcerebellumwm_labels.ravel()) #prints volume of left cerebellum white matter

14
2
[9802412      34   10765      30      43     962     608   15063      54
     159      36      89      56      33      56]
[16762636    14580]


## Function that gets coordinates from labels

In [109]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 1:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [110]:
def get_rs_coord(coordinates):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [111]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Generate midbrain coordinates into a variable

In [112]:
midbrain_int = []
bs = get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels)
lcc = get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels)
lcw = get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels)
rcc = get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels)
rcw = get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels)

midbrain = []
midbrainparts = [brainstem, lcercort, lcerwm, rcercort, rcerwm]

if len(bs) != 1:
    midbrain.append(bs[1])
else:
    midbrain.append(bs[0])

if len(lcc) != 1:
    midbrain.append(lcc[1])
else:
    midbrain.append(lcc[0])

if len(lcw) != 1:
    midbrain.append(lcw[1])
else:
    midbrain.append(lcw[0])

if len(rcc) != 1:
    midbrain.append(rcc[1])
else:
    midbrain.append(rcc[0])

if len(rcw) != 1:
    midbrain.append(rcw[1])
else:
    midbrain.append(rcw[0])

print len(midbrain)



5


## Generate lesion coordinates into a variable

In [113]:
lesions = get_seg_coord(get_rs_coord(get_label_coord(les_labels,n_les_labels)))
lesions_les = get_label_coord(les_labels,n_les_labels)
lesions_seg = lesions
"""
print len(lesions)
print lesions_seg[0][1]
print lesions_les[7]
print lesions_seg[7]
print lesions_les[7][39]
print lesions_seg[7][39]
"""

'\nprint len(lesions)\nprint lesions_seg[0][1]\nprint lesions_les[7]\nprint lesions_seg[7]\nprint lesions_les[7][39]\nprint lesions_seg[7][39]\n'

## Debugging function that finds range of values in brain lesion/segment

In [114]:
test_x = [];test_y=[];test_z=[]
for count in range(len(midbrain[0])):
    test_x.append(midbrain[0][count][0])
    test_y.append(midbrain[0][count][1])
    test_z.append(midbrain[0][count][2])

def range_coord(val_list):
    min_val = min(val_list)
    max_val = max(val_list)
    print min_val,"-", max_val

range_coord(test_x);range_coord(test_y);range_coord(test_z)

109 - 146
148 - 207
83 - 117


## Debugging Function that searches for coordinates in any given brain lesion/segment

In [115]:
def search(coordinates,check_coord):
    number = 0
    for count in range(len(coordinates)):
        if check_coord == coordinates[count]:
            number = count + 1
            print count
    if number == 0:
        print "not available"
    else:
        print "available"

search(midbrain[0],lesions[7][39])
search(midbrain[0],[144, 177, 100])
search(lesions[7],lesions[7][39])

not available
not available
28
39
available


## Infratentorial Lesion Check

In [116]:
les_type = []
inf_num = 0
for count in range(len(lesions)):
    lesion_type = ""
    count4 = 0
    if len(lesions[count]) <= 200:
        for count2 in range(len(lesions[count])):
            halt = ""
            for count3 in range(len(midbrain)):
                for count4 in range(len(midbrain[count3])):
                    if lesions[count][count2] == midbrain[count3][count4]:
                        lesion_type = "infratentorial"
                        halt = "stop"
                    if halt == "stop":
                        continue
                if halt == "stop":
                    continue
            if halt == "stop":
                continue
        if lesion_type == "":
            lesion_type = "subcortical"
        print count, "done, type is:", lesion_type
        les_type.append(lesion_type)
    else:
        print "skipped lesion number", count, "; too large"
        les_type.append("too large")

print les_type

0 done, type is: subcortical
skipped lesion number 1 ; too large
2 done, type is: subcortical
3 done, type is: subcortical
skipped lesion number 4 ; too large
skipped lesion number 5 ; too large
skipped lesion number 6 ; too large
7 done, type is: infratentorial
8 done, type is: subcortical
9 done, type is: subcortical
10 done, type is: subcortical
11 done, type is: subcortical
12 done, type is: subcortical
13 done, type is: subcortical
['subcortical', 'too large', 'subcortical', 'subcortical', 'too large', 'too large', 'too large', 'infratentorial', 'subcortical', 'subcortical', 'subcortical', 'subcortical', 'subcortical', 'subcortical']


## append results to a new list 

In [117]:
results = []
sub_count, inf_count, jux_count, per_count, err_count = 0, 0, 0, 0, 0
for count in range(len(les_type)):
    results.append([count, les_type[count],])
    if les_type[count] == "subcortical":
        sub_count += 1
    elif les_type[count] == "infratentorial":
        inf_count += 1
    elif les_type[count] == "juxtacortical":
        jux_count += 1
    elif les_type[count] == "periventricular":
        per_count += 1
    else:
        err_count += 1
results_dict[subjects[0]] = results, ["total subcortical", sub_count], ["total infratentorial", inf_count], ["total errors", err_count]
print results_dict

{'mse1343': ([[0, 'subcortical'], [1, 'too large'], [2, 'subcortical'], [3, 'subcortical'], [4, 'too large'], [5, 'too large'], [6, 'too large'], [7, 'infratentorial'], [8, 'subcortical'], [9, 'subcortical'], [10, 'subcortical'], [11, 'subcortical'], [12, 'subcortical'], [13, 'subcortical']], ['total subcortical', 9], ['total infratentorial', 1], ['total errors', 4]), 'mseID': ['lesion number and type', 'number of subcortical lesions', 'number of juxtacortical lesions', 'number of periventricular lesions', 'number of infratentorial lesions', 'number of errors']}


## Attempt at understanding "any" function 

In [ ]:
test = any(count for lesions[7][39] in midbrain)
print test

<generator object <genexpr> at 0x7f647eec01e0>


## Debugging Coding Section

In [ ]:
realspace_coordinates = np.dot(les_affine, [average_les[0][0],
                 average_les[0][1], 
                 average_les[0][2],
                 1])
average_les[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine = np.linalg.inv(seg_affine)
freesurfer_matrix = np.dot(inv_seg_affine, [realspace_coordinates[0],
                                            realspace_coordinates[1], 
                                            realspace_coordinates[2],
                                            1])
print average_les[0]; print realspace_coordinates; print freesurfer_matrix

print len(midbrain[0])

les_type = []

for test_count in range(len(lesions)):
    for count in range(len(lesions[test_count])):
        for count2 in range(len(midbrain)):
            halt = ""
            for count3 in range(len(midbrain[count2])):
                if lesions[test_count][count] == midbrain[count2][count3]:
                    inf_num += 1
                    lesion_type = "infratentorial"
                    halt = "stop"
                    break
            if halt == "stop":
                break
    print lesion_type
if lesion_type == "infratentorial":
    les_type.append(lesion_type)
elif lesion_type == "":
    les_type.append("subcortical")

print les_type
print inf_num

[78, 113, 75]
[ 45.23600006  31.30799866  47.93880081   1.        ]
[  81.12500763  134.99999619  113.9375        1.        ]
21730
subcortical


## Function that takes in coordinates for lesions, and outputs average; may not actually use this

In [ ]:
def average_func(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [average_x, average_y, average_z]
        averages_all.append(averages)
        #print count, averages
    return averages_all

In [ ]:
average_les = average_func(get_label_coord(les_labels, n_les_labels))
average_rs = average_func(get_rs_coord(get_label_coord(les_labels, n_les_labels)))
average_seg = average_func(get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels))))

allcoord_les = get_label_coord(les_labels, n_les_labels)
allcoord_rs = get_rs_coord(get_label_coord(les_labels, n_les_labels))
allcoord_seg = get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels)))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]

print average_seg